In [4]:
#process excel file
import win32com.client
import time
import psutil
import os

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
chrome_options = Options()
chrome_options.add_argument(r'--user-data-dir=C:\Users\congthang.van\AppData\Local\Google\Chrome\User Data\Default')  # Path to your Chrome profile
# Optionally, specify your profile directory if you use multiple profiles:
# chrome_options.add_argument(r'--profile-directory=Default')  # Or 'Profile 1', etc.


baseURL = "https://solvnow-smartit.onbmc.com/smartit/app/#/search/"

# Path to your Excel file
# excel_file = r"C:\Users\ManageITVN\OneDrive - Concentrix Corporation\process_pending_ticket\set_pending_new.xlsx"
excel_file = r"C:\Users\congthang.van\OneDrive - Concentrix Corporation\process_pending_ticket\set_pending_new.xlsx" 
# C:\Users\congthang.van\OneDrive - Concentrix Corporation\process_pending_ticket\set_pending_new.xlsx   
ticket_success = []
# Read phrases from Excel
import pandas as pd


tickets=['INC000025031574']
tickets = list(set(tickets))  # Remove duplicates
print(f"Tickets to process: {tickets}")
for ticket in tickets:
        driver = webdriver.Chrome(options=chrome_options)
        driver.get(baseURL+ticket)
        print(f"Processing ticket: {ticket}")
        try:

            time.sleep(20)
            onehub = 'OneHub'   #please change to your desired group
            qtsc = 'HoChiMinh' #please change to your desired group
            flemington = 'VNM-Flemington' #please change to your desired group
            techvalley = 'TechValley' #please change to your desired group
            assign_group = driver.find_element(By.CLASS_NAME, 'search-item-layout__desc')
            # if assign_group.text.find(onehub) ==0:
            #     engineer = "Hoang Tuan Kiet Nguyen"
            # elif assign_group.text.find(qtsc) ==0:
            #     engineer = "Hoang Lam Cao"
            # elif assign_group.text.find(flemington) == 0:
            #     engineer = "Quoc Khai Bui"
            # elif assign_group.text.find(techvalley) == 0:
            #     engineer = "Hoang (Daniel ) Tran"
            # else:
            #     engineer = "Hoang Lam Cao"
            if onehub in assign_group.text:
                engineer = "Hoang Tuan Kiet Nguyen"
            elif qtsc in assign_group.text:
                engineer = "Hoang Lam Cao"
            elif flemington in assign_group.text:
                engineer = "Quoc Khai Bui"
            elif techvalley in assign_group.text:
                engineer = "Hoang (Daniel ) Tran"   
            else:
                engineer = "Hoang Duy Bao Nguyen"
            print(f'''assign engieener to {engineer}''')
        except Exception as e:
            print(f"Failed to find assign engineer: {e}")
            engineer = "Hoang Duy Bao Nguyen"
        ####Process the target link

        try:
            links = driver.find_elements(By.TAG_NAME, "a")
            hrefs = [link.get_attribute("href") for link in links if link.get_attribute("href")]
            
            if hrefs:
                max_href = max(hrefs, key=len)
                print("Link with maximum length:", max_href)
            else:
                print("No href links found.")    
            # Wait for the element by its link text
            driver.get(max_href)
            # pencil_icon = WebDriverWait(driver, 15).until(
            #     EC.element_to_be_clickable((By.XPATH, "//span[contains(@class, 'd-icon-pencil')]"))
            # )
            time.sleep(10)  # Wait for the page to load
            driver.switch_to.frame("pwa-frame")
            time.sleep(15)  # Wait for the iframe to load

            # if flag_send_email == True:
            #     send_troubleshooting_email(ticket, user_email.text)
            # else:
            #     print("No user email found, skipping email notification.")
            element = driver.find_element(By.XPATH, "//div[@id='ar7_data']")
            if element.text == "Assigned":
                            
                try:
                    user_email = driver.find_element(By.ID, "ar1000000048_data")
                    flag_send_email = True
                        # print("ticket is already assigned.")
                except Exception as e:
                    print(f"Skip this ticket {ticket} because it is not user ticket")
                    flag_send_email = False
                if flag_send_email == True:
                    print(user_email.text)
                else:
                    print("No user email found, skipping email notification.")
                edit_buttons = driver.find_element(By.XPATH, "//span[contains(@class, 'd-icon-pencil undefined ng-star-inserted')]")
                edit_buttons.click()
                time.sleep(10)
                
                input_engineer = driver.find_element(By.ID, "ar1000000218")
            
                input_engineer.clear()
                input_engineer.send_keys(engineer)

                time.sleep(3)  # Wait for suggestions to load
                click_engineer = driver.find_element(By.XPATH, "//div[contains(@class, 'textEllipsis') and @data-testmenu]")
                click_engineer.click()
                save_button = driver.find_element(By.XPATH, "//button[@title='Save ticket']")
                save_button.click()
                time.sleep(10)  # Wait for the save action to complete
                edit_buttons = driver.find_element(By.XPATH, "//span[contains(@class, 'd-icon-pencil undefined ng-star-inserted')]")
                edit_buttons.click()
                time.sleep(5)

                status = driver.find_elements(By.CSS_SELECTOR, "div.rx-select__search-button-title")
                print(status[3].text)
            
                if status[3].text == "Assigned":
                    btns = driver.find_elements(By.CSS_SELECTOR, "button.dropdown-toggle")
                    btns[2].click()  # Click the fourth button (index 3)
                    pending_option = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//button[.//adapt-highlight[text()='Pending']]"))
                    )
                    pending_option.click()
                    #change reason pending
                    time.sleep(3)
                    btns = driver.find_elements(By.CSS_SELECTOR, "button.dropdown-toggle")
                    btns[3].click()  # Click the fourth button (index 3)
                    Client_Action_Required = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//button[.//adapt-highlight[text()='Client Action Required']]"))
                    )
                    Client_Action_Required.click()
                    time.sleep(3)
                    save_button = driver.find_element(By.XPATH, "//button[@title='Save ticket']")
                    save_button.click()
                    time.sleep(4)
                    driver.quit()
            
                else:
                    ...
            else:
                print("Element text is not 'Asigned', skipping edit.")
        except Exception as e:
                print(f"Failed to click on the adjust button or select Pending: {e}")
        driver.quit()
        print(f"Ticket {ticket} processed successfully.")

Tickets to process: ['INC000025031574']
Processing ticket: INC000025031574
assign engieener to Quoc Khai Bui
Link with maximum length: https://solvnow-smartit.onbmc.com/smartit/app/#/incidentPV/AGGHJ149JF9FVATCFEICTCFEICOIBI
Skip this ticket INC000025031574 because it is not user ticket
No user email found, skipping email notification.
Assigned
Ticket INC000025031574 processed successfully.
